The Biot-Savart Law:
$$ \vec{B}(\vec{r})={\mu_0\over{4\pi}}\int\frac{ \vec{I}\times\vec{\mathscr{i}}}{\mathscr{i}^3}dl'$$

In [1]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RegularGridInterpolator
from mpl_toolkits.mplot3d.axes3d import Axes3D
from tqdm import tqdm


mu0 = 4 * np.pi * 1e-7

In [2]:
mesh_res_x, mesh_res_y, mesh_res_z = 20, 20, 60 # Mesh resolution
mesh_len_x, mesh_len_y, mesh_len_z = 2, 2, 6

$\vec{r}$: origin to field point; $\vec{r'}$: origin to source point (current element); $\vec{\mathscr{i}}$: Source point to field point

In [3]:
def get_mag_field(X, Y, Z, currents):
    Bx, By, Bz = np.zeros_like(X), np.zeros_like(Y), np.zeros_like(Z)
    prog_bar = tqdm(total=len(currents), desc="Processing B-field")
    for current in currents:
        cr, ci = current[0], current[1] # vec to src; vec of src
        i_vec = np.stack((X,Y,Z), axis=-1) - cr
        i_mag = np.sqrt(np.sum(i_vec**2, axis=-1))
        i_mag[i_mag == 0] = np.finfo(float).eps; # avoid division by zero
        dB = mu0/4/np.pi * np.cross(ci, i_vec) / i_mag[...,None]**3
        Bx += dB[..., 0]
        By += dB[..., 1]
        Bz += dB[..., 2]
        
        prog_bar.update(1)
        # print("Result:", dB)
    prog_bar.close()
    return Bx, By, Bz

def gen_current_coil(r, z, I, res):
    """Generate one revolution of coil at z"""
    angles = np.linspace(0, 2*np.pi, res, endpoint=False)
    ret = []
    for angle in angles:
        ret.append([(r * np.cos(angle), r * np.sin(angle), z),
                   (-I * np.sin(angle), I * np.cos(angle), 0)])
    return np.array(ret)

def get_mag_force(Bx, By, Bz, X, Y, Z, x, y, z, m_dips):
    prog_bar = tqdm(total=len(m_dips), desc="Processing Forces")

    # Precompute gradient
    B_grad = {
        'grad_x': np.stack(np.gradient(Bx, X[:,0,0], Y[0,:,0], Z[0,0,:], edge_order=2), axis=-1),
        'grad_y': np.stack(np.gradient(By, X[:,0,0], Y[0,:,0], Z[0,0,:], edge_order=2), axis=-1),
        'grad_z': np.stack(np.gradient(Bz, X[:,0,0], Y[0,:,0], Z[0,0,:], edge_order=2), axis=-1)
    }
    
    # Get interpolator
    interpolators = {key: RegularGridInterpolator((x,y,z), grad, bounds_error=False, fill_value=0)
                   for key, grad in B_grad.items()}
    
    # Calculate force
    ret = []
    for m_dip in m_dips:
        grad_Bx = interpolators['grad_x'](m_dip[0])[0]
        grad_By = interpolators['grad_y'](m_dip[0])[0]
        grad_Bz = interpolators['grad_z'](m_dip[0])[0]
        
        grad_B = np.array([grad_Bx, grad_By, grad_Bz])
        # print(grad_B)
        ret.append(np.matmul(m_dip[1], grad_B))
        
        prog_bar.update(1)
    
    prog_bar.close()
    return np.array(ret)

    

In [4]:
I_mag = 0.5 # Current

# Shape: [(Rx, Ry, Rz),(Ix, Iy, Iz)]
currents = np.vstack(np.array([gen_current_coil(1, z/20, I_mag, 12) for z in range(-30,30)]))

In [5]:
x = np.linspace(-mesh_len_x / 2, mesh_len_x / 2, mesh_res_x)
y = np.linspace(-mesh_len_y / 2, mesh_len_y / 2, mesh_res_y)
z = np.linspace(-mesh_len_z / 2, mesh_len_z / 2, mesh_res_z)
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

# Calculate
Bx, By, Bz = get_mag_field(X, Y, Z, currents)

Processing B-field: 100%|█████████████████████████████| 720/720 [00:01<00:00, 490.18it/s]


In [24]:
dipoles = []
for iz in range(-14,14):
    for ix in range(-3,3):
        for iy in range(-3,3):
            dipoles.append([(ix/6, iy/6, iz/6), (0,0,1)])
            # dipoles.append([(ix/10, iy/10, 0), (0,0,1)])
dipoles = np.array(dipoles)

In [25]:
forces = get_mag_force(Bx, By, Bz, X, Y, Z, x, y, z, dipoles)

Processing Forces: 100%|███████████████████████████| 1008/1008 [00:00<00:00, 2107.02it/s]


In [28]:
ax = plt.figure().add_subplot(projection='3d')
ax.set_proj_type('ortho')

# Plot the currents
plot_currents, plot_field, plot_force = True, True, True
plot_uniform_scaling = True
psk = 4 # Skip over some vectors to speed up 3D viewer

if plot_currents:
    ax.quiver(
        currents[:,:1,0], currents[:,:1,1], currents[:,:1,2], # Positions
        currents[:,1:,0], currents[:,1:,1], currents[:,1:,2], # Directions
        length=0.05, color='red', normalize=True, label='Current Elements'
    )

m_scale = 1e4 # Magnetic field scaling
if plot_field:
    ax.quiver(X[::psk,::psk,::psk],Y[::psk,::psk,::psk],Z[::psk,::psk,::psk],
              Bx[::psk,::psk,::psk]*m_scale, By[::psk,::psk,::psk]*m_scale, Bz[::psk,::psk,::psk]*m_scale)

f_scale = 2e4 # Force field scaling
if plot_force:
    ax.quiver(
        dipoles[::psk,:1:psk,0], dipoles[::psk,:1:psk,1], dipoles[::psk,:1:psk,2], # Positions
        forces[::psk,:1]*f_scale, forces[::psk,1:2]*f_scale, forces[::psk,2:]*f_scale, # Directions
        color='green', label='Force Elements'
    )

if plot_uniform_scaling:
    ax.set_box_aspect([mesh_res_x, mesh_res_y, mesh_res_z])

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.show()